## Gait Analysis On NTNU Belt Sensor

### About data

The Data is collected on 5 participants where three sensors worn on the belt.The following activities were performed:
1. Walking Downstairs for 15 seconds 
2. Walking straight for 15 seconds 
3. walking Upstairs for 15 seconds 
4. sitting for 6 seconds 
5. Laying  for 6 seconds 
6. standing  for 6 seconds

In [1]:
"""
Gait Analysis On NTNU Belt Sensor
The NTNU belt sensor dataset was collected for detecting activity of daily living (ADLs) on healthy people.
A total of 8 participants performed  activities of daily living: walking, walking downstairs, walking upstairs, sitting,
lying, and standing.Each subject wore three IMU sensors simultaneously at specific body locations around the waist,
including left, right, and middle back of waist, to record activity data at a sampling rate of 100 Hz. 

"""
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pathlib
import csv

C:\Users\adanent\Anaconda3\envs\Annotation_tool\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Read dataset
df = pd.read_csv('C:/Users/adanent/Documents/1_SDDE Seminar/belt_sensor_all.csv')
print(df.shape)
df.head()

(108170, 21)


,Pid,timestamp,S1_Ax,S1_Ay,S1_Az,S1_Gx,S1_Gy,S1_Gz,S2_Ax,S2_Ay,...,S2_Gx,S2_Gy,S2_Gz,S3_Ax,S3_Ay,S3_Az,S3_Gx,S3_Gy,S3_Gz,Label
0,P1_BS,3.00,-84.23,-994.63,8.06,-0.37,-2.62,1.65,311.28,-1061.28,...,2.08,-3.23,-4.33,-68.12,-970.46,150.15,1.83,0.24,0.61,Downstairs
1,P1_BS,3.01,-86.43,-1004.15,16.85,-0.18,-3.17,0.85,301.76,-1024.66,...,6.77,3.05,-3.78,-73.24,-978.52,140.62,0.55,-0.24,0.31,Downstairs
2,P1_BS,3.02,-92.29,-1001.22,9.52,-0.06,-3.72,0.00,290.04,-996.09,...,10.25,8.97,-3.30,-76.17,-983.64,145.02,-0.73,-0.67,0.18,Downstairs
3,P1_BS,3.03,-92.29,-1004.88,19.04,-0.12,-4.39,-0.79,260.74,-964.60,...,11.96,14.10,-2.56,-64.45,-988.77,145.02,-0.73,-1.04,-0.37,Downstairs
4,P1_BS,3.04,-82.03,-1001.22,18.31,-0.31,-5.25,-1.53,229.25,-935.30,...,12.33,18.68,-1.71,-68.12,-982.18,146.48,-0.31,-0.98,-0.43,Downstairs


In [3]:
### Low Pass filter 
from scipy import signal
def butter_lowpass(cutoff, nyq_freq, order=4):
    normal_cutoff = float(cutoff) / nyq_freq
    b, a = signal.butter(order, normal_cutoff, btype='lowpass')
    return b, a

def butter_lowpass_filter(data, cutoff_freq, nyq_freq, order=4):
    b, a = butter_lowpass(cutoff_freq, nyq_freq, order=order)
    y = signal.filtfilt(b, a, data)
    return y

#low pass filter
lowpass_filtred=[]
dfm=df[['S1_Ax', 'S1_Ay', 'S1_Az', 'S1_Gx', 'S1_Gy','S1_Gz', 'S2_Ax', 'S2_Ay', 'S2_Az', 'S2_Gx', 'S2_Gy', 'S2_Gz', 'S3_Ax',
       'S3_Ay', 'S3_Az', 'S3_Gx', 'S3_Gy', 'S3_Gz']]
for column in dfm.columns:# iterate over each column in raw_df
    cutoff_frequency = 15.0
    sample_rate=100
    t_signal=np.array(dfm[column]) #copie the signal values in 1D numpy array
    y = butter_lowpass_filter(t_signal,cutoff_frequency,sample_rate/2) 
    lowpass_filtred.append(y)
lowpass=pd.DataFrame(lowpass_filtred)
lowpass=lowpass.T
lowpass.columns=['S1_Ax', 'S1_Ay', 'S1_Az', 'S1_Gx', 'S1_Gy','S1_Gz', 'S2_Ax', 'S2_Ay', 'S2_Az', 'S2_Gx', 'S2_Gy', 'S2_Gz', 'S3_Ax',
       'S3_Ay', 'S3_Az', 'S3_Gx', 'S3_Gy', 'S3_Gz']
df1=df[['Pid','timestamp']]
df_filtred=pd.concat([df1,lowpass],axis=1)
df_filtred['Label']=df['Label']

In [4]:
df_filtred['Label'].value_counts()

Upstairs      29345
Downstairs    25030
Walking       19440
Standing      12180
Sitting       11265
Laying        10910
Name: Label, dtype: int64

In [5]:
from imblearn.over_sampling import SMOTE

In [6]:
# split data into input and output elements
X = df_filtred[['S1_Ax', 'S1_Ay', 'S1_Az', 'S1_Gx', 'S1_Gy',
       'S1_Gz', 'S2_Ax', 'S2_Ay', 'S2_Az', 'S2_Gx', 'S2_Gy', 'S2_Gz', 'S3_Ax',
       'S3_Ay', 'S3_Az', 'S3_Gx', 'S3_Gy', 'S3_Gz']]
y = df_filtred[['Label']]

oversample = SMOTE()
Xo, yo = oversample.fit_resample(X, y)

df_Smoted=pd.concat([Xo,yo],axis=1)
print(df_Smoted.shape)
df_Smoted.tail(2)

(176070, 19)


,S1_Ax,S1_Ay,S1_Az,S1_Gx,S1_Gy,S1_Gz,S2_Ax,S2_Ay,S2_Az,S2_Gx,S2_Gy,S2_Gz,S3_Ax,S3_Ay,S3_Az,S3_Gx,S3_Gy,S3_Gz,Label
176068,119.140341,-1244.555722,144.227499,-1.112440,-19.619820,-9.849389,68.788399,-1016.769311,133.709294,47.658275,-75.319481,-8.542993,-41.577711,-882.837060,-65.074210,37.903649,-36.405670,-1.104857,Walking
176069,2.013924,-633.919063,182.611411,-12.962844,-2.224018,15.352547,461.837618,-904.749400,64.558267,3.420319,-7.715497,-0.687696,71.733733,-919.918233,404.071775,3.712516,-1.950956,-4.870012,Walking


In [7]:
df_Smoted.groupby('Label').size()

Label
Downstairs    29345
Laying        29345
Sitting       29345
Standing      29345
Upstairs      29345
Walking       29345
dtype: int64

In [8]:
#Label encoding
df_Smoted['Label_id']=df_Smoted['Label']
dff=df_Smoted.replace({'Label_id': {'Downstairs':0,'Walking':1,'Upstairs':2,
                        'Sitting':3,'Laying':4,'Standing':5}})
dff.tail(2)

,S1_Ax,S1_Ay,S1_Az,S1_Gx,S1_Gy,S1_Gz,S2_Ax,S2_Ay,S2_Az,S2_Gx,S2_Gy,S2_Gz,S3_Ax,S3_Ay,S3_Az,S3_Gx,S3_Gy,S3_Gz,Label,Label_id
176068,119.140341,-1244.555722,144.227499,-1.112440,-19.619820,-9.849389,68.788399,-1016.769311,133.709294,47.658275,-75.319481,-8.542993,-41.577711,-882.837060,-65.074210,37.903649,-36.405670,-1.104857,Walking,1
176069,2.013924,-633.919063,182.611411,-12.962844,-2.224018,15.352547,461.837618,-904.749400,64.558267,3.420319,-7.715497,-0.687696,71.733733,-919.918233,404.071775,3.712516,-1.950956,-4.870012,Walking,1


In [9]:
### Normalize/Standardize data
from sklearn.preprocessing import StandardScaler, LabelEncoder
X = dff[['S1_Ax', 'S1_Ay', 'S1_Az', 'S1_Gx', 'S1_Gy','S1_Gz', 'S2_Ax', 'S2_Ay', 'S2_Az', 'S2_Gx', 'S2_Gy', 'S2_Gz', 'S3_Ax',
       'S3_Ay', 'S3_Az', 'S3_Gx', 'S3_Gy', 'S3_Gz']]
y = dff['Label_id']
scaler = StandardScaler()
dx = scaler.fit_transform(X)

df_scaled = pd.DataFrame(data = dx, columns = X.columns)
df_scaled['Label_id'] = y.values
print(df_scaled.shape)
df_scaled.tail()

(176070, 19)


,S1_Ax,S1_Ay,S1_Az,S1_Gx,S1_Gy,S1_Gz,S2_Ax,S2_Ay,S2_Az,S2_Gx,S2_Gy,S2_Gz,S3_Ax,S3_Ay,S3_Az,S3_Gx,S3_Gy,S3_Gz,Label_id
176065,0.679070,0.347546,-0.125695,0.467313,-0.902780,1.030387,-0.062156,0.631200,1.767103,-1.688798,-0.608487,0.238718,0.723421,0.100461,-0.841799,0.359017,-1.166813,-1.359275,1
176066,-0.328980,-0.428495,0.353195,0.082508,-0.320731,0.125342,-0.020888,-0.487139,-0.575148,-0.062198,-0.332779,-0.064034,-0.664166,0.458188,0.789691,0.038124,0.027079,-0.344210,1
176067,-0.046808,-0.777342,-0.206529,-0.084597,0.410207,-0.417811,-0.009746,0.211660,-0.707446,0.336249,-0.441203,-0.920821,0.157545,-0.345076,0.263942,0.835920,0.158520,-0.208450,1
176068,0.219481,-1.115927,-0.100850,-0.061740,-0.708609,-0.454219,0.136106,-0.423843,-0.379211,1.938983,-2.374440,-0.418763,-0.195935,0.020110,-0.174468,1.384478,-1.300465,-0.034479,1
176069,-0.108013,0.801753,0.104447,-0.612535,-0.158705,0.787654,1.276570,-0.062391,-0.688994,0.152786,-0.324152,-0.018797,0.423229,-0.103216,0.958203,0.157377,-0.137377,-0.234131,1


In [10]:
## Segmentation (window Preparation)
import scipy.stats as stats
Fs = 100
segment_size = 500 # fs*3 # window size=n_time steps=frame size
step_size = 50  #  Fs/2 # step size is related to window overlap
n_features = 18

def get_segments(df, segment_size, step_size):

    segments = []
    labels = []
    
    for i in range(0, len(df) - segment_size - 1, step_size):
        S1Ax = df_scaled['S1_Ax'].values[i: i + segment_size]
        S1Ay = df_scaled['S1_Ay'].values[i: i + segment_size]
        S1Az = df_scaled['S1_Az'].values[i: i + segment_size]
        S1Gx = df_scaled['S1_Gx'].values[i: i + segment_size]
        S1Gy = df_scaled['S1_Gy'].values[i: i + segment_size]
        S1Gz = df_scaled['S1_Gz'].values[i: i + segment_size]
        
        S2Ax = df_scaled['S2_Ax'].values[i: i + segment_size]
        S2Ay = df_scaled['S2_Ay'].values[i: i + segment_size]
        S2Az = df_scaled['S2_Az'].values[i: i + segment_size]
        S2Gx = df_scaled['S2_Gx'].values[i: i + segment_size]
        S2Gy = df_scaled['S2_Gy'].values[i: i + segment_size]
        S2Gz = df_scaled['S2_Gz'].values[i: i + segment_size]
        
        
        S3Ax = df_scaled['S3_Ax'].values[i: i + segment_size]
        S3Ay = df_scaled['S3_Ay'].values[i: i + segment_size]
        S3Az = df_scaled['S3_Az'].values[i: i + segment_size]
        S3Gx = df_scaled['S3_Gx'].values[i: i + segment_size]
        S3Gy = df_scaled['S3_Gy'].values[i: i + segment_size]
        S3Gz = df_scaled['S3_Gz'].values[i: i + segment_size]
        
          # Retrieve the most often used label in this segment
        label = stats.mode(df_scaled['Label_id'][i: i + segment_size])[0][0] ## [0][0] shows the current sequence
        
        #We used the dstack() to ensure that each array is stacked in such a way that
        #the features are separated in the third dimension, as we would prefer.
        segments.append(np.dstack([S1Ax, S1Ay, S1Az,S1Gx,S1Gy,S1Gz,S2Ax,S2Ay,S2Az, S2Gx,S2Gy,S2Gz,S3Ax,S3Ay,S3Az,S3Gx,S3Gy,S3Gz]))
        labels.append(label)

    # Bring the segments into a better shape
    segments = np.asarray(segments).reshape(-1, segment_size, n_features)
    labels = np.asarray(labels)
        
    return segments, labels

In [11]:
X, y = get_segments(df_scaled, segment_size, step_size)

In [12]:
X.shape, y.shape

((3512, 500, 18), (3512,))

In [13]:
##Feature Extraction from each original variables (columns)
import scipy.stats as stats
from scipy.stats import skew,kurtosis
from IPython.display import clear_output

def feature_extraction(df):
    mean=[]
    std=[]
    var=[]
    minn=[]
    maxx=[]
    median=[]
    skew=[]
    kurtos=[]
    for i in df:
        mn=np.mean(i,axis=0) # calculate mean to each window, for each axis on each sensor 
        st=np.std(i,axis=0)  # calculate standard deviation to each window, for each axis on each sensor
        vr=np.var(i,axis=0)  # calculate variance to each window, for each axis on each sensor 
        mi=np.min(i,axis=0) 
        mx=np.max(i,axis=0) 
        med=np.median(i,axis=0) 
        sk=stats.skew(i, axis=0, bias=True)
        kur=stats.kurtosis(i, axis=0, bias=True)
        
        mean.append(mn) 
        std.append(st)
        var.append(vr) 
        minn.append(mi)
        maxx.append(mx)
        median.append(med)
        skew.append(sk)
        kurtos.append(kur)
        
    dfmean=pd.DataFrame(mean)
    dfstd=pd.DataFrame(std)
    dfvar=pd.DataFrame(var)
    dfmi=pd.DataFrame(minn)
    dfmx=pd.DataFrame(maxx)
    dfmed=pd.DataFrame(median)
    dfsk=pd.DataFrame(skew)
    dfkur=pd.DataFrame(kurtos)
    
    dft=pd.concat([dfmean,dfstd,dfvar,dfmi,dfsk,dfmx,dfmed,dfkur],axis=1)
    #dft.columns=
    return dft 

In [14]:
dfx_fs=feature_extraction(X)
print(dfx_fs.shape)
print(y.shape)
dfx_fs.head(2)

(3512, 144)
(3512,)


,0,1,2,3,4,5,6,7,8,9,...,8,9,10,11,12,13,14,15,16,17
0,-0.397034,-0.283468,-0.519682,0.063504,-0.146499,0.091945,0.147618,-0.289075,-0.259552,-0.008853,...,1.235052,1.154215,0.620728,1.857914,12.937483,12.337065,15.212485,12.395119,2.292955,2.739161
1,-0.428467,-0.292877,-0.460285,-0.005907,-0.110987,0.044313,0.142294,-0.342769,-0.261170,-0.030100,...,0.520902,4.444391,0.042805,1.818767,9.740758,7.004329,9.524459,8.413464,1.456014,2.070813


In [15]:
dfx_fs['Label']=y
print(dfx_fs.shape)
dfx_fs.tail()

(3512, 145)


,0,1,2,3,4,5,6,7,8,9,...,9,10,11,12,13,14,15,16,17,Label
3507,-0.294448,-0.367104,-0.193349,-0.126045,-0.320607,0.142683,0.230697,-0.311834,-0.357052,0.031021,...,0.543489,1.021769,1.260784,3.935109,1.510808,0.594362,6.332025,0.902487,1.847894,1
3508,-0.287805,-0.372442,-0.208746,-0.153694,-0.338369,0.143892,0.248982,-0.342468,-0.370964,0.044546,...,0.766374,1.047316,1.164329,4.191209,2.441217,0.562483,6.679231,0.960857,2.031652,1
3509,-0.309360,-0.375525,-0.206178,-0.138163,-0.329482,0.124322,0.254883,-0.355221,-0.375190,0.036104,...,0.949151,1.064752,0.879622,4.440310,2.638793,0.545726,6.547498,1.073927,1.594375,1
3510,-0.295073,-0.371428,-0.204113,-0.103813,-0.336648,0.101327,0.266592,-0.358666,-0.351520,-0.013384,...,1.005344,1.261030,1.346055,4.475997,2.624933,0.416566,5.729235,1.021613,1.615770,1
3511,-0.296423,-0.395351,-0.206449,-0.105318,-0.339552,0.087072,0.292012,-0.351076,-0.363664,-0.009639,...,0.947491,1.297412,1.111629,4.441710,2.684504,0.292883,5.603747,1.009181,1.738590,1


In [16]:
#dfx_fs.to_csv('belt_sensor_all_GP.csv')

In [17]:
### Train-Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dfx_fs, y, test_size = 0.3, random_state = 7, stratify = y)

##import libraries 
#from sklearn.ensemble import RandomForestClassifier 
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import datetime

#model train and evaluation
svm=SVC()
svm.fit(X_train, y_train)
ypred_train=svm.predict(X_train)
ypred_test=svm.predict(X_test)

C:\Users\adanent\Anaconda3\envs\Annotation_tool\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\adanent\Anaconda3\envs\Annotation_tool\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\adanent\Anaconda3\envs\Annotation_tool\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [18]:
#dfx_fs.to_csv('belt_sensor_all_GP.csv')

In [19]:
#Print Accuracy
accuracy_train = accuracy_score(y_train, ypred_train)*100
accuracy_test = accuracy_score(y_test,ypred_test )*100
#print('Accuracy_train: {}%'.format(accuracy_train))
print('Aaccuracy_test: {}%'.format(accuracy_test))

Aaccuracy_test: 88.89943074003796%
